# FIXED COUPON BOND EXAMPLE APPLE CORP

This is a  bond analysis based on example in https://data.bloomberglp.com/bat/sites/3/2017/07/SF-2017_Paul-Fjeldsted.pdf

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.bonds import *

####################################################################
# FINANCEPY BETA Version 0.260 - This build:  22 Nov 2022 at 12:35 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



# Define the Bond

In [3]:
issue_date = Date(13, 5, 2010)
maturity_date = Date(13, 5, 2022)
coupon = 0.027
freq_type = FrequencyTypes.SEMI_ANNUAL
accrual_type = DayCountTypes.THIRTY_E_360
face = ONE_MILLION

In [4]:
bond = Bond(issue_date, maturity_date, coupon, freq_type, accrual_type, face)

In [5]:
clean_price = 101.581564  # if face is 1 then this must be 0.99780842

You can get information about the bond using the print method.

In [6]:
print(bond)

OBJECT TYPE: Bond
ISSUE DATE: 13-MAY-2010
MATURITY DATE: 13-MAY-2022
COUPON (%): 2.7
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
ACCRUAL TYPE: DayCountTypes.THIRTY_E_360
FACE AMOUNT: 1000000


## Bond Cash Flows

We first need to set the settlement date of the bond. 

In [7]:
settlement_date = Date(21, 7, 2017)

In [10]:
print(bond.coupon_dates(settlement_date))

 13-NOV-2017     13500.00 
 13-MAY-2018     13500.00 
 13-NOV-2018     13500.00 
 13-MAY-2019     13500.00 
 13-NOV-2019     13500.00 
 13-MAY-2020     13500.00 
 13-NOV-2020     13500.00 
 13-MAY-2021     13500.00 
 13-NOV-2021     13500.00 
 13-MAY-2022   1013500.00 



The convention is to use these dates for yield calculations even if some fall on weekends.

## Bond Yield Measures

Current yield is the coupon over the price

In [11]:
bond.current_yield(clean_price)*100.0

2.657962620067555

Yield to maturity using UK convention

In [12]:
bond.yield_to_maturity(settlement_date, clean_price, YTMCalcType.UK_DMO)*100

2.3499999793942994

Yield to maturity using US Street convention

In [13]:
bond.yield_to_maturity(settlement_date, clean_price, YTMCalcType.US_STREET)*100

2.3499999793942994

Yield to maturity using US Treasury convention

In [14]:
bond.yield_to_maturity(settlement_date, clean_price, YTMCalcType.US_TREASURY)*100

2.3496418128944643

## Accrued Interest

For consistency let's fix the yield calculation convention to be US Street

In [15]:
yieldConvention = YTMCalcType.US_STREET

In [16]:
ytm = bond.yield_to_maturity(settlement_date, clean_price, yieldConvention)

Dirty price is the clean price plus accrued interest

In [17]:
print("Dirty Price = = %8.6f" %  bond.dirty_price_from_ytm(settlement_date, ytm, yieldConvention))

Dirty Price = = 102.091564


In [18]:
print("Clean Price = %8.6f" % bond.clean_price_from_ytm(settlement_date, ytm, yieldConvention))

Clean Price = 101.581564


Accrued interest is accrued from previous coupon date to settlement date

In [19]:
print("Previous coupon date is ", bond._pcd)

Previous coupon date is  13-MAY-2017


In [20]:
print("Settlement date is ", settlement_date)

Settlement date is  21-JUL-2017


The amount of accrued interest is 

In [21]:
print("Accrued = %12.2f"% bond._accrued_interest)

Accrued =      5100.00


This is based on the following number of days of accrual (for some conventions this is not the actual number of days between last coupon and settlement)

In [22]:
print("Accrued Days = ", bond._accrued_days)

Accrued Days =  68


In [23]:
print("Principal = %12.2f" % bond.principal(settlement_date, ytm, yieldConvention))

Principal =   1015815.64


## Bond Risk Measures

The bond dollar duration is the actual derivative with respect to the yield. It is the bond price change for a 1bp drop in the yield-to-maturity divided by 1bp.

In [24]:
duration = bond.dollar_duration(settlement_date, ytm, yieldConvention)
print("Dollar Duration = ", duration)

Dollar Duration =  456.56933693578594


Modified Duration divides the dollar duration by the full price of the bond

In [25]:
modified_duration = bond.modified_duration(settlement_date, ytm, yieldConvention)
print("Modified Duration = ", modified_duration)

Modified Duration =  4.472155377458872


Macauley Duration multiplies the dollar duration by (1+y/f) and divides by the full price

In [26]:
macauley_duration = bond.macauley_duration(settlement_date, ytm, yieldConvention)
print("Macauley Duration = ", macauley_duration)

Macauley Duration =  4.524703202683254


Convexity is the second derivative of the bond price with respect to the yield-to-maturity

In [27]:
conv = bond.convexity_from_ytm(settlement_date, ytm, yieldConvention)
print("Convexity = ", conv)

Convexity =  0.23024091234084085


Copyright (c) 2020 Dominic O'Kane